# SHA256
https://www.programmingalgorithms.com/algorithm/sha256

In [58]:
MAX_UINT32 = 0xffffffff

def DBL_INT_ADD(ab,c):
    (a,b) = ab
    if b > MAX_UINT32 - c:
        a+=1
    b = (b+c)%(1<<32)
    return (a, b)
    
def ROTLEFT(a,b):
    return ((a << (b)) & MAX_UINT32) | (a >> (32-b))
def ROTRIGHT(a,b):
    return (a >> (b)) | ((a << (32-b)) & MAX_UINT32)

def CH(x,y,z):
    return (x & y) ^ ((x^MAX_UINT32) & z)

def MAJ(x,y,z):
    return (x & y) ^ (x & z) ^ (y & z)

def EP0(x):
    return ROTRIGHT(x,2) ^ ROTRIGHT(x,13) ^ ROTRIGHT(x,22)
def EP1(x):
    return ROTRIGHT(x,6) ^ ROTRIGHT(x,11) ^ ROTRIGHT(x,25)

def SIG0(x):
    return ROTRIGHT(x,7) ^ ROTRIGHT(x,18) ^ ((x) >> 3)
def SIG1(x):
    return ROTRIGHT(x,17) ^ ROTRIGHT(x,19) ^ ((x) >> 10)


k = [
    0x428a2f98,0x71374491,0xb5c0fbcf,0xe9b5dba5,0x3956c25b,0x59f111f1,0x923f82a4,0xab1c5ed5,
    0xd807aa98,0x12835b01,0x243185be,0x550c7dc3,0x72be5d74,0x80deb1fe,0x9bdc06a7,0xc19bf174,
    0xe49b69c1,0xefbe4786,0x0fc19dc6,0x240ca1cc,0x2de92c6f,0x4a7484aa,0x5cb0a9dc,0x76f988da,
    0x983e5152,0xa831c66d,0xb00327c8,0xbf597fc7,0xc6e00bf3,0xd5a79147,0x06ca6351,0x14292967,
    0x27b70a85,0x2e1b2138,0x4d2c6dfc,0x53380d13,0x650a7354,0x766a0abb,0x81c2c92e,0x92722c85,
    0xa2bfe8a1,0xa81a664b,0xc24b8b70,0xc76c51a3,0xd192e819,0xd6990624,0xf40e3585,0x106aa070,
    0x19a4c116,0x1e376c08,0x2748774c,0x34b0bcb5,0x391c0cb3,0x4ed8aa4a,0x5b9cca4f,0x682e6ff3,
    0x748f82ee,0x78a5636f,0x84c87814,0x8cc70208,0x90befffa,0xa4506ceb,0xbef9a3f7,0xc67178f2
];


class sha256:
    def __init__(self):
        self.data = [0]*64
        self.datalen = 0;
        self.bitlen = (0,0)
        self.state = [
            0x6a09e667,
            0xbb67ae85,
            0x3c6ef372,
            0xa54ff53a,
            0x510e527f,
            0x9b05688c,
            0x1f83d9ab,
            0x5be0cd19
        ]
    def SHA256Transform(self,data):
        m = [0]*64
        for i in range(0,16):
            j = i*4
            m[i] = (data[j] << 24) | (data[j + 1] << 16) | (data[j + 2] << 8) | (data[j + 3])
            m[i] &= MAX_UINT32
            
        for i in range(16,64):
            m[i] = SIG1(m[i - 2]) + m[i - 7] + SIG0(m[i - 15]) + m[i - 16];
            m[i] &= MAX_UINT32
            
        [a,b,c,d,e,f,g,h] = self.state
        
        for i in range(64):
            t1 = h + EP1(e) + CH(e, f, g) + k[i] + m[i]
            t2 = EP0(a) + MAJ(a, b, c)
            t1 = t1 & MAX_UINT32
            t2 = t2 & MAX_UINT32
                       
            h = g
            g = f
            f = e
            e = (d + t1) & MAX_UINT32
            d = c
            c = b
            b = a
            a = (t1 + t2) & MAX_UINT32
            
        self.state[0] += a & MAX_UINT32
        self.state[1] += b & MAX_UINT32
        self.state[2] += c & MAX_UINT32
        self.state[3] += d & MAX_UINT32
        self.state[4] += e & MAX_UINT32
        self.state[5] += f & MAX_UINT32
        self.state[6] += g & MAX_UINT32
        self.state[7] += h & MAX_UINT32
    
    def SHA256Update(self, data):
        for i in data:
            self.data[self.datalen] = i;
            self.datalen+=1;
            if self.datalen != 64:
                continue
            self.SHA256Transform(self.data)
            self.bitlen = DBL_INT_ADD(self.bitlen, 512)
            self.datalen = 0

            
    def SHA256Final(self):

        i = self.datalen;

        if self.datalen < 56:
            self.data[i] = 0x80;
            i+=1
            while i < 56:
                self.data[i] = 0x00
                i+=1
        else:
            self.data[i] = 0x80

            while i<64:
                self.data[i] = 0x00
                i+=1
            self.SHA256Transform(self.data)
            self.data = [0]*64
            
        self.bitlen = DBL_INT_ADD(self.bitlen, self.datalen * 8)
        self.data[63] = self.bitlen[1]
        self.data[62] = self.bitlen[1] >> 8
        self.data[61] = self.bitlen[1] >> 16
        self.data[60] = self.bitlen[1] >> 24
        self.data[59] = self.bitlen[0]
        self.data[58] = self.bitlen[0] >> 8
        self.data[57] = self.bitlen[0] >> 16
        self.data[56] = self.bitlen[0] >> 2
        
        self.SHA256Transform(self.data);
        
        h = [0]*32
        for i in range(4):
            h[i] = (self.state[0] >> (24 - i * 8)) & 0x000000ff;
            h[i + 4] = (self.state[1] >> (24 - i * 8)) & 0x000000ff;
            h[i + 8] = (self.state[2] >> (24 - i * 8)) & 0x000000ff;
            h[i + 12] = (self.state[3] >> (24 - i * 8)) & 0x000000ff;
            h[i + 16] = (self.state[4] >> (24 - i * 8)) & 0x000000ff;
            h[i + 20] = (self.state[5] >> (24 - i * 8)) & 0x000000ff;
            h[i + 24] = (self.state[6] >> (24 - i * 8)) & 0x000000ff;
            h[i + 28] = (self.state[7] >> (24 - i * 8)) & 0x000000ff;
            
        return bytes(h)


def sha256sum(data):
    a = sha256()
    a.SHA256Update(data)
    return a.SHA256Final()
    


In [60]:
import hashlib
sha256sum(b'a'*57).hex(),hashlib.sha256(b'a').digest().hex()

('0f45e858fbc4176cdf4e411f88281edefc390ae5afe7df0f44cd9297f0a64580',
 '0f45e858fbc4176cdf4e411f88281edefc390ae5afe7df0f44cd9297f0a64580')